In [5]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [7]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [157]:
%%sql
DROP TABLE IF EXISTS Branch;
CREATE TABLE IF NOT EXISTS Branch (
    branch_id INTEGER PRIMARY KEY,
    phone TEXT
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [159]:
%%sql
DROP TABLE IF EXISTS Customer;
CREATE TABLE IF NOT EXISTS Customer (
    cust_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT,
    membership_status TEXT CHECK (membership_status IN ('Active', 'Suspended')) DEFAULT 'Active'
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [161]:
%%sql
DROP TABLE IF EXISTS Employee;
CREATE TABLE IF NOT EXISTS Employee (
    employee_id INTEGER PRIMARY KEY,
    branch_id INTEGER,
    first_name TEXT,
    last_name TEXT,
    role TEXT CHECK (role IN ('Librarian', 'Assistant', 'Event Coordinator', 'Volunteer')),
    email TEXT,
    phone TEXT,
    FOREIGN KEY (branch_id) REFERENCES Branch(branch_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [163]:
%%sql
DROP TABLE IF EXISTS Room;
CREATE TABLE IF NOT EXISTS Room (
    room_id INTEGER PRIMARY KEY,
    branch_id INTEGER,
    location TEXT,
    capacity INTEGER CHECK (capacity > 0),
    FOREIGN KEY (branch_id) REFERENCES Branch(branch_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [165]:
%%sql
DROP TABLE IF EXISTS Item;
CREATE TABLE IF NOT EXISTS Item (
    item_id INTEGER PRIMARY KEY,
    title TEXT,
    author TEXT,
    type TEXT CHECK (type IN ('Print', 'Online', 'Magazine', 'Journal', 'CD', 'Record')),
    branch_id INTEGER,
    status TEXT CHECK (status IN ('Available', 'Borrowed', 'Reserved', 'Pending')),
    dewey_decimal TEXT,
    borrow_date DATE,
    due_date DATE,
    FOREIGN KEY (branch_id) REFERENCES Branch(branch_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [167]:
%%sql
DROP TABLE IF EXISTS Borrowed;
CREATE TABLE IF NOT EXISTS Borrowed (
    cust_id INTEGER,
    item_id INTEGER PRIMARY KEY,
    FOREIGN KEY (cust_id) REFERENCES Customer(cust_id),
    FOREIGN KEY (item_id) REFERENCES Item(item_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [169]:
%%sql
DROP TABLE IF EXISTS Fine;
CREATE TABLE IF NOT EXISTS Fine (
    fine_id INTEGER PRIMARY KEY AUTOINCREMENT,
    cust_id INTEGER,
    item_id INTEGER,
    amount FLOAT CHECK (amount >= 0),
    paid_status BOOLEAN DEFAULT 0,
    FOREIGN KEY (cust_id) REFERENCES Customer(cust_id),
    FOREIGN KEY (item_id) REFERENCES Item(item_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [171]:
%%sql
DROP TABLE IF EXISTS Event;
CREATE TABLE IF NOT EXISTS Event (
    event_id INTEGER PRIMARY KEY,
    room_id INTEGER,
    branch_id INTEGER,
    event_name TEXT,
    event_type TEXT,
    date DATE,
    time TIME,
    recommended_audience TEXT,
    FOREIGN KEY (room_id) REFERENCES Room(room_id),
    FOREIGN KEY (branch_id) REFERENCES Branch(branch_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [173]:
%%sql
DROP TABLE IF EXISTS Registered;
CREATE TABLE IF NOT EXISTS Registered (
    cust_id INTEGER,
    event_id INTEGER,
    PRIMARY KEY (cust_id, event_id),
    FOREIGN KEY (cust_id) REFERENCES Customer(cust_id),
    FOREIGN KEY (event_id) REFERENCES Event(event_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [175]:
%%sql
DROP TRIGGER IF EXISTS set_item_borrowed;
CREATE TRIGGER set_item_borrowed
AFTER INSERT ON Borrowed
FOR EACH ROW
BEGIN
    UPDATE Item
    SET 
        status = 'Borrowed',
        borrow_date = DATE('now'),
        due_date = DATE('now', '+14 days')
    WHERE item_id = NEW.item_id;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [177]:
%%sql
DROP TRIGGER IF EXISTS process_return_and_fine;
CREATE TRIGGER process_return_and_fine
AFTER DELETE ON Borrowed
FOR EACH ROW
BEGIN
    INSERT INTO Fine (cust_id, item_id, amount, paid_status)
    SELECT 
        OLD.cust_id, 
        OLD.item_id, 
        10.00, 
        0
    WHERE (SELECT due_date FROM Item WHERE item_id = OLD.item_id) < DATE('now');

    UPDATE Item
    SET 
        status = 'Available',
        borrow_date = NULL,
        due_date = NULL
    WHERE item_id = OLD.item_id;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [179]:
%%sql
DROP TRIGGER IF EXISTS check_membership_status;
CREATE TRIGGER check_membership_status
BEFORE INSERT ON Borrowed
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN (SELECT membership_status FROM Customer WHERE cust_id = NEW.cust_id) != 'Active'
        THEN RAISE(ABORT, 'Customer must be Active to borrow items')
    END;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [181]:
%%sql
DROP TRIGGER IF EXISTS update_membership_status;
CREATE TRIGGER update_membership_status
AFTER INSERT ON Fine
FOR EACH ROW
BEGIN
    UPDATE Customer
    SET membership_status = 'Suspended'
    WHERE cust_id = NEW.cust_id
    AND (SELECT COUNT(*) FROM Fine WHERE cust_id = NEW.cust_id AND paid_status = 0) >= 3;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [183]:
%%sql
DROP TRIGGER IF EXISTS check_event_capacity;
CREATE TRIGGER check_event_capacity
BEFORE INSERT ON Registered
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN ((SELECT COUNT(*) FROM Registered WHERE event_id = NEW.event_id) + 1) >
        (SELECT capacity FROM Room JOIN Event ON Room.room_id = Event.room_id WHERE Event.event_id = NEW.event_id)
        THEN RAISE(ABORT, 'Event has reached maximum capacity')
    END;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [187]:
%%sql
DROP TRIGGER IF EXISTS update_membership_status_active;
CREATE TRIGGER update_membership_status_active
AFTER UPDATE ON Fine
FOR EACH ROW
BEGIN
    UPDATE Customer
    SET membership_status = 'Active'
    WHERE cust_id = NEW.cust_id
    AND (SELECT COUNT(*) FROM Fine WHERE cust_id = NEW.cust_id AND paid_status = 0) <= 3;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [189]:
%%sql
DROP TRIGGER IF EXISTS check_item_status;
CREATE TRIGGER check_item_status
BEFORE INSERT ON Borrowed
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN (SELECT status FROM Item WHERE item_id = NEW.item_id) != 'Available'
        THEN RAISE(ABORT, 'Item not available for borrowing')
    END;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [195]:
%%sql
INSERT INTO Branch (branch_id, phone) VALUES
(1, '555-0001'),
(2, '555-0002'),
(3, '555-0003'),
(4, '555-0004'),
(5, '555-0005'),
(6, '555-0006'),
(7, '555-0007'),
(8, '555-0008'),
(9, '555-0009'),
(10, '555-0010');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [197]:
%%sql
INSERT INTO Customer (cust_id, first_name, last_name, email, phone, membership_status) VALUES
(1, 'John', 'Doe', 'john@example.com', '555-1001', 'Active'),
(2, 'Jane', 'Smith', 'jane@example.com', '555-1002', 'Active'),
(3, 'Bob', 'Johnson', 'bob@example.com', '555-1003', 'Active'),
(4, 'Alice', 'Brown', 'alice@example.com', '555-1004', 'Active'),
(5, 'Charlie', 'Davis', 'charlie@example.com', '555-1005', 'Active'),
(6, 'Eve', 'Wilson', 'eve@example.com', '555-1006', 'Active'),
(7, 'Frank', 'Moore', 'frank@example.com', '555-1007', 'Active'),
(8, 'Grace', 'Taylor', 'grace@example.com', '555-1008', 'Active'),
(9, 'Henry', 'Anderson', 'henry@example.com', '555-1009', 'Active'),
(10, 'Ivy', 'Thomas', 'ivy@example.com', '555-1010', 'Active');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [199]:
%%sql
INSERT INTO Employee (employee_id, branch_id, first_name, last_name, role, email, phone) VALUES
(1, 1, 'Alice', 'Smith', 'Librarian', 'alice@library.com', '555-2001'),
(2, 1, 'Bob', 'Jones', 'Assistant', 'bob@library.com', '555-2002'),
(3, 2, 'Charlie', 'Brown', 'Event Coordinator', 'charlie@library.com', '555-2003'),
(4, 2, 'Diana', 'Green', 'Volunteer', 'diana@library.com', '555-2004'),
(5, 3, 'Evan', 'White', 'Librarian', 'evan@library.com', '555-2005'),
(6, 3, 'Fiona', 'Black', 'Assistant', 'fiona@library.com', '555-2006'),
(7, 4, 'George', 'King', 'Event Coordinator', 'george@library.com', '555-2007'),
(8, 4, 'Hannah', 'Queen', 'Volunteer', 'hannah@library.com', '555-2008'),
(9, 5, 'Ian', 'Prince', 'Librarian', 'ian@library.com', '555-2009'),
(10, 5, 'Jill', 'Princess', 'Assistant', 'jill@library.com', '555-2010');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [201]:
%%sql
INSERT INTO Room (room_id, branch_id, location, capacity) VALUES
(1, 1, 'Floor 1, Room A', 30),
(2, 1, 'Floor 2, Room B', 25),
(3, 2, 'Main Hall', 50),
(4, 3, 'Study Room 1', 20),
(5, 4, 'Conference Room', 40),
(6, 5, 'Media Room', 5),
(7, 6, 'Quiet Zone', 30),
(8, 7, 'Children Section', 25),
(9, 8, 'Auditorium', 100),
(10, 9, 'Seminar Room', 35);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [205]:
%%sql
INSERT INTO Item (item_id, title, author, type, branch_id, status, dewey_decimal) VALUES
(1, 'The Great Novel', 'Author A', 'Print', 1, 'Available', '123.45'),
(2, 'Science Today', 'Author B', 'Magazine', 2, 'Available', '234.56'),
(3, 'History of Time', 'Author C', 'Journal', 3, 'Available', '345.67'),
(4, 'Learn SQL', 'Author D', 'Online', 4, 'Available', '456.78'),
(5, 'Music Classics', 'Author E', 'CD', 5, 'Available', '567.89'),
(6, 'Art of Programming', 'Author F', 'Print', 1, 'Available', '678.90'),
(7, 'National Geographic', 'Author G', 'Magazine', 2, 'Available', '789.01'),
(8, 'Math Basics', 'Author H', 'Print', 3, 'Available', '890.12'),
(9, 'Java Tutorial', 'Author I', 'Online', 4, 'Available', '901.23'),
(10, 'Rock Hits', 'Author J', 'Record', 5, 'Available', '012.34'),
(11, 'To Kill a Mockingbird', 'Harper Lee', 'Print', 1, 'Available', '813.54'),
(12, '1984', 'George Orwell', 'Print', 2, 'Available', '823.912'),
(13, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Print', 3, 'Available', '813.52'),
(14, 'Pride and Prejudice', 'Jane Austen', 'Print', 4, 'Available', '823.7'),
(15, 'The Hobbit', 'J.R.R. Tolkien', 'Print', 5, 'Available', '823.912'),
(16, 'The Catcher in the Rye', 'J.D. Salinger', 'Print', 1, 'Available', '813.54'),
(17, 'Animal Farm', 'George Orwell', 'Print', 2, 'Available', '823.912'),
(18, 'Brave New World', 'Aldous Huxley', 'Print', 3, 'Available', '823.912'),
(19, 'The Lord of the Rings', 'J.R.R. Tolkien', 'Print', 4, 'Available', '823.912'),
(20, 'The Chronicles of Narnia', 'C.S. Lewis', 'Print', 5, 'Available', '823.912'),
(21, 'Fahrenheit 451', 'Ray Bradbury', 'Print', 1, 'Available', '813.54'),
(22, 'The Grapes of Wrath', 'John Steinbeck', 'Print', 2, 'Available', '813.52'),
(23, 'Moby Dick', 'Herman Melville', 'Print', 3, 'Available', '813.3'),
(24, 'War and Peace', 'Leo Tolstoy', 'Print', 4, 'Available', '891.73'),
(25, 'The Odyssey', 'Homer', 'Print', 5, 'Available', '883.01'),
(26, 'Crime and Punishment', 'Fyodor Dostoevsky', 'Print', 1, 'Pending', '891.73'),
(27, 'The Brothers Karamazov', 'Fyodor Dostoevsky', 'Print', 2, 'Pending', '891.73'),
(28, 'Anna Karenina', 'Leo Tolstoy', 'Print', 3, 'Pending', '891.73'),
(29, 'The Divine Comedy', 'Dante Alighieri', 'Print', 4, 'Pending', '851.1'),
(30, 'Les Misérables', 'Victor Hugo', 'Print', 5, 'Pending', '843.7'),
(31, 'Don Quixote', 'Miguel de Cervantes', 'Print', 1, 'Available', '863'),
(32, 'The Picture of Dorian Gray', 'Oscar Wilde', 'Print', 2, 'Available', '823.8'),
(33, 'Wuthering Heights', 'Emily Brontë', 'Print', 3, 'Available', '823.8'),
(34, 'Jane Eyre', 'Charlotte Brontë', 'Print', 4, 'Available', '823.8'),
(35, 'Great Expectations', 'Charles Dickens', 'Print', 5, 'Available', '823.8'),
(36, 'The Count of Monte Cristo', 'Alexandre Dumas', 'Print', 1, 'Available', '843.7'),
(37, 'Frankenstein', 'Mary Shelley', 'Print', 2, 'Available', '823.7'),
(38, 'Dracula', 'Bram Stoker', 'Print', 3, 'Available', '823.8'),
(39, 'The Adventures of Huckleberry Finn', 'Mark Twain', 'Print', 4, 'Available', '813.4'),
(40, 'The Scarlet Letter', 'Nathaniel Hawthorne', 'Print', 5, 'Available', '813.3');

Running query in 'sqlite:///library.db'

40 rows affected.

++
||
++
++

In [235]:
%%sql
    DELETE FROM Borrowed;
    DELETE FROM Fine;
    UPDATE Customer SET membership_status = 'Active';
INSERT INTO Borrowed (cust_id, item_id) VALUES
(1, 1),
(1, 2),
(1, 3),
(4, 4),
(5, 5);
UPDATE Item SET due_date = '2023-01-01' WHERE item_id IN (1, 2, 3, 4, 5);
DELETE FROM Borrowed WHERE item_id IN (1, 2, 3, 4, 5);

INSERT INTO Borrowed (cust_id, item_id) VALUES
(4, 6),
(5, 7),
(3, 8),
(2, 9),
(7, 10);
UPDATE Item SET due_date = '2023-01-01' WHERE item_id IN (6, 7, 8, 9, 10);
DELETE FROM Borrowed WHERE item_id IN (6, 7, 8, 9, 10);

Running query in 'sqlite:///library.db'

10 rows affected.

5 rows affected.

5 rows affected.

5 rows affected.

5 rows affected.

5 rows affected.

5 rows affected.

++
||
++
++

In [217]:
%%sql
INSERT INTO Event (event_id, room_id, branch_id, event_name, event_type, date, time, recommended_audience) VALUES
(1, 1, 1, 'Book Club', 'Literary', '2023-10-15', '14:00', 'Adults'),
(2, 3, 2, 'Science Fair', 'Educational', '2023-10-16', '10:00', 'All Ages'),
(3, 4, 3, 'Coding Workshop', 'Educational', '2023-10-17', '15:30', 'Teens'),
(4, 5, 4, 'Art Class', 'Art', '2023-10-18', '11:00', 'Children'),
(5, 6, 5, 'Music Concert', 'Entertainment', '2023-10-19', '18:00', 'All Ages'),
(6, 7, 6, 'Lecture Series', 'Educational', '2023-10-20', '16:00', 'Adults'),
(7, 8, 7, 'Story Time', 'Children', '2023-10-21', '10:30', 'Children'),
(8, 9, 8, 'Film Screening', 'Entertainment', '2023-10-22', '19:00', 'Adults'),
(9, 10, 9, 'Tech Talk', 'Educational', '2023-10-23', '13:00', 'Teens'),
(10, 2, 1, 'Poetry Reading', 'Literary', '2023-10-24', '17:00', 'All Ages');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [219]:
%%sql
INSERT INTO Registered (cust_id, event_id) VALUES
(1, 1),
(2, 2),
(3, 3),
(4, 4),
(5, 5),
(6, 6),
(7, 7),
(8, 8),
(9, 9),
(10, 10);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [241]:
%%sql
INSERT INTO Borrowed VALUES
    (6, 2),
    (3, 7),
    (3, 5),
    (5, 3),
    (3, 10),
    (2, 1),
    (4, 4),
    (4, 6),
    (5, 8),
    (5, 9)

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [221]:
%%sql
SELECT * FROM Item

Running query in 'sqlite:///library.db'

item_id,title,author,type,branch_id,status,dewey_decimal,borrow_date,due_date
1,The Great Novel,Author A,Print,1,Available,123.45,None,None
2,Science Today,Author B,Magazine,2,Available,234.56,None,None
3,History of Time,Author C,Journal,3,Available,345.67,None,None
4,Learn SQL,Author D,Online,4,Available,456.78,None,None
5,Music Classics,Author E,CD,5,Available,567.89,None,None
6,Art of Programming,Author F,Print,1,Available,678.90,None,None
7,National Geographic,Author G,Magazine,2,Available,789.01,None,None
8,Math Basics,Author H,Print,3,Available,890.12,None,None
9,Java Tutorial,Author I,Online,4,Available,901.23,None,None
10,Rock Hits,Author J,Record,5,Available,012.34,None,None


In [223]:
%%sql
SELECT * FROM Customer

Running query in 'sqlite:///library.db'

cust_id,first_name,last_name,email,phone,membership_status
1,John,Doe,john@example.com,555-1001,Suspended
2,Jane,Smith,jane@example.com,555-1002,Active
3,Bob,Johnson,bob@example.com,555-1003,Active
4,Alice,Brown,alice@example.com,555-1004,Suspended
5,Charlie,Davis,charlie@example.com,555-1005,Suspended
6,Eve,Wilson,eve@example.com,555-1006,Active
7,Frank,Moore,frank@example.com,555-1007,Active
8,Grace,Taylor,grace@example.com,555-1008,Active
9,Henry,Anderson,henry@example.com,555-1009,Active
10,Ivy,Thomas,ivy@example.com,555-1010,Active


In [239]:
%%sql
SELECT * FROM Fine F LEFT JOIN Customer C ON F.cust_id = C.cust_id

Running query in 'sqlite:///library.db'

fine_id,cust_id,item_id,amount,paid_status,cust_id_1,first_name,last_name,email,phone,membership_status
21,1,1,10.0,0,1,John,Doe,john@example.com,555-1001,Suspended
22,1,2,10.0,0,1,John,Doe,john@example.com,555-1001,Suspended
23,1,3,10.0,0,1,John,Doe,john@example.com,555-1001,Suspended
24,4,4,10.0,0,4,Alice,Brown,alice@example.com,555-1004,Active
25,5,5,10.0,0,5,Charlie,Davis,charlie@example.com,555-1005,Active
26,4,6,10.0,0,4,Alice,Brown,alice@example.com,555-1004,Active
27,5,7,10.0,0,5,Charlie,Davis,charlie@example.com,555-1005,Active
28,3,8,10.0,0,3,Bob,Johnson,bob@example.com,555-1003,Active
29,2,9,10.0,0,2,Jane,Smith,jane@example.com,555-1002,Active
30,7,10,10.0,0,7,Frank,Moore,frank@example.com,555-1007,Active


In [243]:
%%sql
SELECT * FROM Borrowed LEFT JOIN Item ON Borrowed.item_id = Item.item_id LEFT JOIN Customer ON Borrowed.cust_id = Customer.cust_id

Running query in 'sqlite:///library.db'

cust_id,item_id,item_id_1,title,author,type,branch_id,status,dewey_decimal,borrow_date,due_date,cust_id_1,first_name,last_name,email,phone,membership_status
2,1,1,The Great Novel,Author A,Print,1,Borrowed,123.45,2025-04-02,2025-04-16,2,Jane,Smith,jane@example.com,555-1002,Active
6,2,2,Science Today,Author B,Magazine,2,Borrowed,234.56,2025-04-02,2025-04-16,6,Eve,Wilson,eve@example.com,555-1006,Active
5,3,3,History of Time,Author C,Journal,3,Borrowed,345.67,2025-04-02,2025-04-16,5,Charlie,Davis,charlie@example.com,555-1005,Active
4,4,4,Learn SQL,Author D,Online,4,Borrowed,456.78,2025-04-02,2025-04-16,4,Alice,Brown,alice@example.com,555-1004,Active
3,5,5,Music Classics,Author E,CD,5,Borrowed,567.89,2025-04-02,2025-04-16,3,Bob,Johnson,bob@example.com,555-1003,Active
4,6,6,Art of Programming,Author F,Print,1,Borrowed,678.90,2025-04-02,2025-04-16,4,Alice,Brown,alice@example.com,555-1004,Active
3,7,7,National Geographic,Author G,Magazine,2,Borrowed,789.01,2025-04-02,2025-04-16,3,Bob,Johnson,bob@example.com,555-1003,Active
5,8,8,Math Basics,Author H,Print,3,Borrowed,890.12,2025-04-02,2025-04-16,5,Charlie,Davis,charlie@example.com,555-1005,Active
5,9,9,Java Tutorial,Author I,Online,4,Borrowed,901.23,2025-04-02,2025-04-16,5,Charlie,Davis,charlie@example.com,555-1005,Active
3,10,10,Rock Hits,Author J,Record,5,Borrowed,012.34,2025-04-02,2025-04-16,3,Bob,Johnson,bob@example.com,555-1003,Active


In [245]:
%%sql
SELECT * FROM Item ORDER BY branch_id

Running query in 'sqlite:///library.db'

item_id,title,author,type,branch_id,status,dewey_decimal,borrow_date,due_date
1,The Great Novel,Author A,Print,1,Borrowed,123.45,2025-04-02,2025-04-16
6,Art of Programming,Author F,Print,1,Borrowed,678.90,2025-04-02,2025-04-16
11,To Kill a Mockingbird,Harper Lee,Print,1,Available,813.54,None,None
16,The Catcher in the Rye,J.D. Salinger,Print,1,Available,813.54,None,None
21,Fahrenheit 451,Ray Bradbury,Print,1,Available,813.54,None,None
26,Crime and Punishment,Fyodor Dostoevsky,Print,1,Pending,891.73,None,None
31,Don Quixote,Miguel de Cervantes,Print,1,Available,863,None,None
36,The Count of Monte Cristo,Alexandre Dumas,Print,1,Available,843.7,None,None
2,Science Today,Author B,Magazine,2,Borrowed,234.56,2025-04-02,2025-04-16
7,National Geographic,Author G,Magazine,2,Borrowed,789.01,2025-04-02,2025-04-16


Running query in 'sqlite:///library.db'

++
||
++
++